# Twitter Sentiment Classification: Positive vs. Negative

In [4]:
import pandas as pd

df_train = pd.read_csv('../data/twitter_sentiment_train.csv')
df_test  = pd.read_csv('../data/twitter_sentiment_test.csv')

In [5]:
int_to_label = {1: 'Positive', 0: 'Negative'}

In [6]:
df_train.head(2)

,text,label
0,"""If Paul Dunne wins the Open tomorrow as an am...",1
1,Spreading the word about our newest twilight m...,1
